In [1]:
def savings(cls):
    cls.acc_type = 'savings'
    return cls


def checking(cls):
    cls.acc_type = 'checking'
    return cls

In [3]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

#Bank1Savings  = savings(Bank1Savings)

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

In [4]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'acc_type': 'savings'})

In [5]:
Bank2Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'acc_type': 'checking'})

In [6]:
def acc_type(type_):
    def dec(cls):
        cls.acc_type = type_
        return cls
    return dec

In [7]:
@acc_type('Savings')
class Bank1Savings:
    pass

In [8]:
@acc_type('Checking')
class Bank1Checking:
    pass

In [9]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
              '__doc__': None,
              'acc_type': 'Savings'})

In [10]:
Bank1Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
              '__doc__': None,
              'acc_type': 'Checking'})

In [11]:
def hello(cls):
    cls.hello = lambda self:f'{self} says hello'
    return cls

In [13]:
@hello
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name

In [14]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [15]:
p = Person('Guido')

In [16]:
p.hello()

'Guido says hello'

In [17]:
from functools import wraps

In [18]:
def func_log(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log:{fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

In [19]:
class Person:
    @func_log
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @func_log
    def greet(self):
        return f'hello, my name is {self.name}, and i m {self.age} old'

In [20]:
p = Person('john', 78)

Log:Person.__init__((<__main__.Person object at 0x00000245CEA3EA60>, 'john', 78), {}) = None


In [21]:
p.greet()

Log:Person.greet((<__main__.Person object at 0x00000245CEA3EA60>,), {}) = hello, my name is john, and i m 78 old


'hello, my name is john, and i m 78 old'

In [23]:
def class_log(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('dec', cls, name)
            setattr(cls, name, func_log(obj))
    return cls

In [24]:
@class_log
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'hello, my name is {self.name}, and i m {self.age} old'

dec <class '__main__.Person'> __init__
dec <class '__main__.Person'> greet


In [25]:
p = Person('john', 78)

Log:Person.__init__((<__main__.Person object at 0x00000245CE6D7D60>, 'john', 78), {}) = None


In [26]:
p.greet()

Log:Person.greet((<__main__.Person object at 0x00000245CE6D7D60>,), {}) = hello, my name is john, and i m 78 old


'hello, my name is john, and i m 78 old'

In [27]:
@class_log
class Person:
    @staticmethod
    def static_method():
        print('static method invoked')
        
    @classmethod
    def cls_method(cls):
        print(f'cls_method invoked for {cls}')
        
    def Inst_method(self):
        print(f'inst method for {self}')

dec <class '__main__.Person'> Inst_method


In [28]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'static_method': <staticmethod at 0x245ce0ebd60>,
              'cls_method': <classmethod at 0x245ce0fd8b0>,
              'Inst_method': <function __main__.Person.Inst_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [32]:
callable(Person.__dict__['static_method'])

False

In [33]:
class Person:
    @staticmethod
    @func_log
    def static_mthod():
        pass

In [34]:
Person.static_mthod()

Log:Person.static_mthod((), {}) = None


In [35]:
class Person:
    @func_log
    @staticmethod
    def static_mthod():
        pass

In [36]:
Person.static_mthod()

TypeError: 'staticmethod' object is not callable

In [37]:
class Person:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def class_method(cls):
        pass

In [38]:
type(Person.__dict__['static_method'])

staticmethod

In [39]:
type(Person.__dict__['class_method'])

classmethod

In [40]:
Person.__dict__['static_method'].__func__

<function __main__.Person.static_method()>

In [46]:
def cls_log(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('dec callable', cls, name)
            original_fn = obj
            dec_fn = func_log(original_fn)
            setattr(cls, name, dec_fn)
        elif isinstance(obj, staticmethod):
            print('dec static method', cls, name)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = staticmethod(dec_fn)
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            print('dec class method', cls, name)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = classmethod(dec_fn)
            setattr(cls, name, method)
            
    return cls

In [48]:
@cls_log
class Person:
    @staticmethod
    def static_method(a,b):
        print('static_method called', a,b)
        
    @classmethod
    def class_method(cls,a,b):
        print('class_method called', a,b)
        
    def inst_method(self,a,b):
        print('insta method called', a,b)

dec static method <class '__main__.Person'> static_method
dec class method <class '__main__.Person'> class_method
dec callable <class '__main__.Person'> inst_method


In [49]:
Person.static_method(10,20)

static_method called 10 20
Log:Person.static_method((10, 20), {}) = None


In [50]:
Person.class_method(10,20)

class_method called 10 20
Log:Person.class_method((<class '__main__.Person'>, 10, 20), {}) = None


In [51]:
p = Person()

In [53]:
p.inst_method(10,20)

insta method called 10 20
Log:Person.inst_method((<__main__.Person object at 0x00000245CE0FD0D0>, 10, 20), {}) = None


In [54]:
@class_log
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

dec <class '__main__.Person'> __init__


In [55]:
type(Person.__dict__['name'])

property

In [56]:
isinstance(Person.__dict__['name'], property)

True

In [57]:
prop= Person.__dict__['name']

In [58]:
prop.fget

<function __main__.Person.name(self)>

In [59]:
prop.fset, prop.fdel

(None, None)

In [60]:
def cls_log(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('dec callable', cls, name)
            original_fn = obj
            dec_fn = func_log(original_fn)
            setattr(cls, name, dec_fn)
        elif isinstance(obj, staticmethod):
            print('dec static method', cls, name)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = staticmethod(dec_fn)
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            print('dec class method', cls, name)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = classmethod(dec_fn)
            setattr(cls, name, method)
        
        elif isinstance(obj, property):
            print('dec prop', cls, name)
            if obj.fget:
                obj = obj.getter(func_log(obj.fget))
            if obj.fset:
                obj = obj.setter(func_log(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_log(obj.fdel))
            setattr(cls, name, obj)
            
                
            
    return cls




In [62]:
@cls_log
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

dec callable <class '__main__.Person'> __init__
dec prop <class '__main__.Person'> name


In [63]:
p = Person('david')

Log:Person.__init__((<__main__.Person object at 0x00000245CFFC3E80>, 'david'), {}) = None


In [64]:
p.name

Log:Person.name((<__main__.Person object at 0x00000245CFFC3E80>,), {}) = david


'david'

In [65]:
@cls_log
class Person:
    class Other:
        def __call__(self):
            print('called inst of other')
    other = Other()

dec callable <class '__main__.Person'> Other
dec callable <class '__main__.Person'> other


In [66]:
Person.Other

<function __main__.Person.Other()>

In [67]:
Person.other

<function __main__.func_log.<locals>.inner()>

In [68]:
import inspect

In [70]:
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    def ins_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other() 

In [71]:
keys = ('static_method', 'cls_method', 'ins_method', 'name', '__add__', 'Other', 'other')
inspect_fns = ('isroutine', 'ismethod', 'isfunction', 'isbuiltin', 'ismethoddescriptor')

In [75]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_fns)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_fns:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	ins_method   	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


In [76]:
def cls_log(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod):
            print('dec static method', cls, name)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = staticmethod(dec_fn)
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            print('dec class method', cls, name)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = classmethod(dec_fn)
            setattr(cls, name, method)
        
        elif isinstance(obj, property):
            print('dec prop', cls, name)
            if obj.fget:
                obj = obj.getter(func_log(obj.fget))
            if obj.fset:
                obj = obj.setter(func_log(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_log(obj.fdel))
            setattr(cls, name, obj)
            
        elif  inspect.isroutine(obj):
            print('dec callable', cls, name)
            original_fn = obj
            dec_fn = func_log(original_fn)
            setattr(cls, name, dec_fn)
            
                
            
    return cls




In [78]:
@cls_log
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    def ins_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    @cls_log
    class Other:
        def __call__(self):
            pass
        
    other = Other() 

dec callable <class '__main__.MyClass.Other'> __call__
dec static method <class '__main__.MyClass'> static_method
dec class method <class '__main__.MyClass'> cls_method
dec callable <class '__main__.MyClass'> ins_method
dec prop <class '__main__.MyClass'> name
dec callable <class '__main__.MyClass'> __add__


In [83]:
def cls_log(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, classmethod) or isinstance(obj, staticmethod):
            print(f'decorating: {name}')
            type_ = type(obj)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = type_(dec_fn)
            setattr(cls, name, method)
            
        
       
        
        elif isinstance(obj, property):
            print('dec prop', cls, name)
            if obj.fget:
                obj = obj.getter(func_log(obj.fget))
            if obj.fset:
                obj = obj.setter(func_log(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_log(obj.fdel))
            setattr(cls, name, obj)
            
        elif  inspect.isroutine(obj):
            print('dec callable', cls, name)
            original_fn = obj
            dec_fn = func_log(original_fn)
            setattr(cls, name, dec_fn)
    return cls

In [84]:
@cls_log
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    def ins_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    @cls_log
    class Other:
        def __call__(self):
            pass
        
    other = Other() 

dec callable <class '__main__.MyClass.Other'> __call__
decorating: static_method
decorating: cls_method
dec callable <class '__main__.MyClass'> ins_method
dec prop <class '__main__.MyClass'> name
dec callable <class '__main__.MyClass'> __add__


In [85]:
def cls_log(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, classmethod) or isinstance(obj, staticmethod):
            print(f'decorating: {name}')
            type_ = type(obj)
            original_fn = obj.__func__
            dec_fn = func_log(original_fn)
            method = type_(dec_fn)
            setattr(cls, name, method)

        elif isinstance(obj, property):
            print('dec prop', cls, name)
            methods = (('fget', 'getter'), ('fset', 'setter'), ('fdel', 'deleter'))
            for prop, method in methods:
                if getattr(obj, prop):
                    obj = getattr(obj, method)(func_log(getattr(obj, prop)))
                    setattr(cls, name, obj)
        elif  inspect.isroutine(obj):
            print('dec callable', cls, name)
            original_fn = obj
            dec_fn = func_log(original_fn)
            setattr(cls, name, dec_fn)
    return cls

In [86]:
@cls_log
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    def ins_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    @cls_log
    class Other:
        def __call__(self):
            pass
        
    other = Other() 

dec callable <class '__main__.MyClass.Other'> __call__
decorating: static_method
decorating: cls_method
dec callable <class '__main__.MyClass'> ins_method
dec prop <class '__main__.MyClass'> name
dec callable <class '__main__.MyClass'> __add__
